In [2]:
import sys
sys.path.append('../')
import wrangle

import nlp

In [3]:
from sklearn.decomposition import LatentDirichletAllocation, NMF

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
import pandas as pd
import numpy as np

### Test

In [9]:
def assign_topic(input_column, max_df=.8, min_df=2, stop_words='english', ngram_range=(1,3), n_components=3):
    input_column = input_column.dropna().apply(nlp.basic_clean)
    input_column = input_column.apply(nlp.lemmatize)
    count_vect = CountVectorizer(max_df=max_df, min_df=min_df, stop_words=stop_words, ngram_range=ngram_range)
    doc_term_matrix = count_vect.fit_transform(input_column.values.astype('U'))
    LDA = LatentDirichletAllocation(n_components=n_components, random_state=42)
    LDA.fit(doc_term_matrix)
    lda_H = LDA.transform(doc_term_matrix)
    topic_doc_df = pd.DataFrame(lda_H)
    return topic_doc_df.idxmax(axis=1)

In [10]:
topic_test = assign_topic(df.how_pick_events)

In [12]:
test_column = df.how_pick_events.dropna().apply(nlp.basic_clean)

In [13]:
test_column = test_column.apply(nlp.lemmatize)

In [14]:
test_column.shape

(644,)

In [11]:
topic_test

0      0
1      0
2      1
3      2
4      1
      ..
639    0
640    1
641    2
642    2
643    2
Length: 644, dtype: int64

In [8]:
df.how_pick_events

resp_id
284                                                  NaN
288    Content, cost, timing (can I get away at this ...
294    - time for travel (having two small children]\...
281    My professional development budget allows me t...
280                                                  NaN
                             ...                        
869    Is it esteemed/lasting, is it relevant to my i...
879    Distance is a concern - leaving my young kids,...
136    For meetups, i will attend if the topic is int...
74     Location is a big factor. I try to make it in ...
2      US travel, prefer West coast. Price a huge con...
Name: how_pick_events, Length: 726, dtype: object

### End Test

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as esw

stopWords = ['yes', 'sort', 've', 'thing', 'try', '50', 'conference'] + list(esw) 

In [ ]:
def set_stop_words(stop_words):
    from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as esw
    stopWords = stop_words + list(esw) 
    return stopWords

## SPACING

In [6]:
df, data_dict = wrangle.wrangle_data()

In [ ]:
execs = nlp.create_persona_corpus(df, 5)

In [ ]:
execs.big_answer.apply(nlp.find_subjectivity).mean()

In [ ]:
nlp.show_persona_keywords(execs.big_answer, 0.8, 3, (1,3), 15)

## Creating topic feature

In [ ]:
input_column = df.recommendations

In [ ]:
doc_term_matrix, count_vect = nlp.create_wordcount_matrix(input_column, ngram=(1,3))

In [ ]:
LDA = LatentDirichletAllocation(n_components=3, random_state=42)
LDA.fit(doc_term_matrix)

In [ ]:
LDA.fit(matrix).transform(matrix)

In [ ]:
LDA.transform(doc_term_matrix)

In [ ]:
lda_H = LDA.transform(doc_term_matrix)

In [ ]:
input_column

In [ ]:
lda_H.shape

In [ ]:
lda_H[0]

In [ ]:
np.amax(lda_H, axis=0)

In [ ]:
np.argsort(lda_H[0]).min()

In [ ]:
np.sort(lda_H[0])

In [ ]:
topic_doc_series = pd.Series(lda_H)

In [ ]:
topic_doc_df.iloc[0]

In [ ]:
topic_doc_df.idxmax(axis=1)

In [ ]:
len(LDA.components_[0])

In [ ]:
def assign_topic(input_column):
    count_vect = CountVectorizer(max_df=.8, min_df=2, stop_words='english', ngram_range=(1,3))
    doc_term_matrix = count_vect.fit_transform(input_column.values.astype('U'))
    LDA = LatentDirichletAllocation(n_components=3, random_state=42)
    LDA.fit(doc_term_matrix)
    lda_H = LDA.transform(doc_term_matrix)
    topic_doc_df = pd.DataFrame(lda_H)
    return topic_doc_df.idxmax(axis=1)

In [ ]:
assign_topic(input_column)